In [1]:
## No elimine esta celda, ya que permite actualizar los archivos .py
## sin tener que reiniciar el Kernel de Jupyter.
%load_ext autoreload
%autoreload 2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.datasets import ImageFolder
import numpy as np
import torch
import timm
from train import automate_training, find_best_model, testing, FrozenNet, run_experiment
from transforms import Transforms
from utils import plot_images, set_seed, count_instances_per_class
import os
from torch.utils.data import random_split
import warnings


warnings.filterwarnings("ignore", message = "Palette images with Transparency expressed in bytes should be converted to RGBA images")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

set_seed()


In [ ]:
train_transforms = A.Compose(
    [A.Resize(256, 256), A.ToFloat(), ToTensorV2()]
)
validation_transforms = A.Compose(
    [A.Resize(256, 256), A.ToFloat(), ToTensorV2()]
)


train_data = ImageFolder(
    "house_plant_species/train",
    transform = Transforms(train_transforms),
)
validation_data = ImageFolder(
    "house_plant_species/validation",
    transform = Transforms(validation_transforms),
)



print(f"Elementos en Entrenamiento: {len(train_data)}")
print(f"Elementos en Validación: {len(validation_data)}")
print(f"There are {len(train_data.classes)} classes!")
#

# Llamar a las funciones
class_counts = count_instances_per_class(train_data)

# Calcular estadísticas
class_instance_counts = np.array(list(class_counts.values()))
mean_instances = np.mean(class_instance_counts)
std_instances = np.std(class_instance_counts)
min_instances = np.min(class_instance_counts)
max_instances = np.max(class_instance_counts)

# # Mostrar resultados
# print("\nInstancias por clase:\n")
# for class_name, count in class_counts.items():
#     print(f"Clase {class_name}: {count} imágenes\n===========================================")

print("\nEstadísticas de las instancias por clase:\n")
print(f"Promedio de instancias:\n============================== {mean_instances:.2f}")
print(f"Desviación estándar:\n============================== {std_instances:.2f}")
print(f"Cantidad mínima de instancias:\n============================== {min_instances}")
print(f"Cantidad máxima de instancias:\n============================== {max_instances}")

### Best architecture is **efficientnet_b5.sw_in12k** obtained with:

###### "color_jitter", "horizontal_flip", "brightness_contrast", "blur"
###### Trivial fc_layers => Lazylinear(512) => Linear(512, num_clases)
###### Resize to 256 x 256 images
###### ToFloat() and not Normalize

#### Same architecture for FrozenNet, see new params just below

In [ ]:
run_experiment(
    selected_transforms = ["center_crop"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "new_functions",
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 10},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 437  # Aquí defines dinámicamente el tamaño de `resize`
)


### More augments and Dropout
###### self.fc_layers = nn.Sequential(
###### nn.LazyLinear(512),   # Primera capa con menos parámetros
###### nn.ReLU(),            # Activación no lineal para mayor capacidad expresiva
###### nn.Dropout(0.3),      # Dropout para prevenir el sobreajuste (30% de unidades apagadas)
###### nn.Linear(512, 256),  # Segunda capa intermedia más pequeña
###### nn.ReLU(),            # Activación no lineal
###### nn.Dropout(0.3),      # Dropout adicional para regularizar
###### nn.Linear(256, self.num_classes)  # Capa final de salida
)


In [ ]:
run_experiment(
    selected_transforms = ["center_crop", "color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "lots_augments",
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 437  # Aquí defines dinámicamente el tamaño de `resize`
)

#### Less augments with batch normalization

 ###### self.fc_layers = nn.Sequential(
 ######     nn.LazyLinear(512),          # Primera capa totalmente conectada
 ######     nn.BatchNorm1d(512),         # Normalización por lotes para la salida de la capa
 ######     nn.ReLU(),                   # Activación no lineal
 ######     nn.Dropout(0.3),             # Dropout para prevenir el sobreajuste

 ######     nn.Linear(512, 256),         # Segunda capa totalmente conectada más pequeña
 ######     nn.BatchNorm1d(256),         # Normalización por lotes
 ######     nn.ReLU(),                   # Activación no lineal
 ######     nn.Dropout(0.3),             # Dropout adicional para regularizar
 ######     nn.Linear(256, self.num_classes)  # Capa final de salida
 ###### )

In [ ]:
run_experiment(
    selected_transforms = ["center_crop", "color_jitter", "horizontal_flip", "brightness_contrast"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "batch_less",
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 437  # Aquí defines dinámicamente el tamaño de `resize`
)

#### Only CenterCrop with BatchNorm and Dropout


In [2]:
run_experiment(
    selected_transforms = ["center_crop"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "center_bn_drp",
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 437  # Aquí defines dinámicamente el tamaño de `resize`
)


Running experiment: efficientnet_b5.sw_in12k__center_bn_drp
Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


  2%|▏         | 1/50 [02:39<2:10:33, 159.87s/it]

Epoch: 1/50 - Time: 159.68s - Train Loss: 1.4836, Train Accuracy: 0.7443 - Validation Loss: 0.5568, Validation Accuracy: 0.9054
Model checkpoint saved at epoch 1


  4%|▍         | 2/50 [05:20<2:08:06, 160.13s/it]

...

Epoch: 16/50 - Time: 165.10s - Train Loss: 0.0371, Train Accuracy: 0.9900 - Validation Loss: 0.2443, Validation Accuracy: 0.9388


 32%|███▏      | 16/50 [46:42<1:39:16, 175.18s/it]

Epoch: 17/50 - Time: 165.61s - Train Loss: 0.0384, Train Accuracy: 0.9891 - Validation Loss: 0.2557, Validation Accuracy: 0.9324
Early stopping triggered at epoch 7. Best Validation Loss: 0.2212


elapsed_time,▁▁▂▂▂▂▄▃▃▄▅█▁▂▂▃▃
epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
train_acc,▁▆▇▇█████████████
train_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▆█▇▇▇▇██▇▇▇▆▆▇█▆
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▂▂▂▁▂
elapsed_time,165.60526
epoch,17
train_acc,0.98909
train_loss,0.0384
val_acc,0.93239



=== Evaluación en el conjunto de prueba ===
Test Accuracy: 0.94150
Results to save: {'architecture': 'efficientnet_b5.sw_in12k__center_bn_drp', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 7, 'best_val_loss': 0.22122688015140052, 'best_train_loss': 0.07095233075039671, 'max_val_acc': 0.9406121373176575, 'test_accuracy': 0.9414991140365601, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__center_bn_drp


#### Same architecture but with lots of augs

In [2]:
run_experiment(
    selected_transforms = ["center_crop", "color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "lots_aug_bn_drp", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 437  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__lots_aug_bn_drp
Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1/50 - Time: 159.91s - Train Loss: 1.5699, Train Accuracy: 0.7210 - Validation Loss: 0.5366, Validation Accuracy: 0.9095


  2%|▏         | 1/50 [02:40<2:10:45, 160.12s/it]

...

Epoch: 14/50 - Time: 165.71s - Train Loss: 0.0885, Train Accuracy: 0.9743 - Validation Loss: 0.2031, Validation Accuracy: 0.9434


 28%|██▊       | 14/50 [41:37<1:47:02, 178.40s/it]

Epoch: 15/50 - Time: 165.53s - Train Loss: 0.0815, Train Accuracy: 0.9752 - Validation Loss: 0.2057, Validation Accuracy: 0.9479
Early stopping triggered at epoch 5. Best Validation Loss: 0.1897


elapsed_time,▁▁▄▄▅▂▂▄▃▄▄▃█▃▃
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▇▇▇▇█████████
train_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▆▇▇▆▇███▇▇▇▇█
val_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁
elapsed_time,165.52949
epoch,15
train_acc,0.97522
train_loss,0.08153
val_acc,0.94792



=== Evaluación en el conjunto de prueba ===
Test Accuracy: 0.95247
Results to save: {'architecture': 'efficientnet_b5.sw_in12k__lots_aug_bn_drp', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 9, 'best_val_loss': 0.1896509627636144, 'best_train_loss': 0.12131835857803035, 'max_val_acc': 0.9483782649040222, 'test_accuracy': 0.9524679780006409, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__lots_aug_bn_drp


### Same architecture but with weight decay of 1e-4

In [2]:
run_experiment(
    selected_transforms = ["center_crop", "color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "lots_aug_bn_drp_wd", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 437  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__lots_aug_bn_drp_wd
Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


  2%|▏         | 1/50 [02:39<2:10:07, 159.34s/it]

Epoch: 1/50 - Time: 159.15s - Train Loss: 1.5762, Train Accuracy: 0.7160 - Validation Loss: 0.5315, Validation Accuracy: 0.9063
Model checkpoint saved at epoch 1
Epoch: 2/50 - Time: 159.08s - Train Loss: 0.5131, Train Accuracy: 0.9027 - Validation Loss: 0.3046, Validation Accuracy: 0.9333


  4%|▍         | 2/50 [05:18<2:07:29, 159.36s/it]

...

Epoch: 13/50 - Time: 227.25s - Train Loss: 0.0978, Train Accuracy: 0.9714 - Validation Loss: 0.2189, Validation Accuracy: 0.9420


 26%|██▌       | 13/50 [41:17<1:57:32, 190.61s/it]

Epoch: 14/50 - Time: 203.79s - Train Loss: 0.0897, Train Accuracy: 0.9745 - Validation Loss: 0.2122, Validation Accuracy: 0.9447
Early stopping triggered at epoch 8. Best Validation Loss: 0.1906


elapsed_time,▁▁▁▂▁▁▂▂▃▃▃▆█▆
epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
train_acc,▁▆▇▇▇▇████████
train_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▆▇▆▇▇██▇▇▆▆▇▇
val_loss,█▃▂▂▁▁▁▁▁▁▁▁▂▁
elapsed_time,203.79434
epoch,14
train_acc,0.97448
train_loss,0.08968
val_acc,0.94472



=== Evaluación en el conjunto de prueba ===
Test Accuracy: 0.95064
Results to save: {'architecture': 'efficientnet_b5.sw_in12k__lots_aug_bn_drp_wd', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 8, 'best_val_loss': 0.190603235979443, 'best_train_loss': 0.13104292344764845, 'max_val_acc': 0.9479214549064636, 'test_accuracy': 0.9506398439407349, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__lots_aug_bn_drp_wd


In [ ]:
run_experiment(
    selected_transforms = ["center_crop", "color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "lots_aug_bn_drp_wd", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 437  # Aquí defines dinámicamente el tamaño de `resize`
)

In [ ]:
run_experiment(
    selected_transforms = ["center_crop", "color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "lots_aug_bn_drp_VAL", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 32, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 437  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__lots_aug_bn_drp_VAL
Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: joseflores1 (benjaminrromo-universidad-adolfo-ib-ez). Use `wandb login --relogin` to force relogin


  2%|▏         | 1/50 [02:46<2:16:22, 166.98s/it]

Epoch: 1/50 - Time: 166.80s - Train Loss: 1.5507, Train Accuracy: 0.7269 - Validation Loss: 0.5087, Validation Accuracy: 0.9148
Model checkpoint saved at epoch 1


  4%|▍         | 2/50 [05:50<2:21:15, 176.58s/it]

...

Epoch: 14/50 - Time: 173.21s - Train Loss: 0.0870, Train Accuracy: 0.9750 - Validation Loss: 0.2037, Validation Accuracy: 0.9477


 28%|██▊       | 14/50 [45:42<1:57:31, 195.89s/it]

Epoch: 15/50 - Time: 183.30s - Train Loss: 0.0945, Train Accuracy: 0.9717 - Validation Loss: 0.1985, Validation Accuracy: 0.9433
Early stopping triggered at epoch 9. Best Validation Loss: 0.1888


elapsed_time,▁▅▃▆▆█▆█▂▆▅▄▅▃▅
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▇▇▇▇█████████
train_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▆▇▇▆▇▆█▆▇▆▇▇▆
val_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁
elapsed_time,183.30485
epoch,15
train_acc,0.97171
train_loss,0.09452
val_acc,0.94335


Results to save: {'architecture': 'efficientnet_b5.sw_in12k__lots_aug_bn_drp_VAL', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 9, 'best_val_loss': 0.18883399100064538, 'best_train_loss': 0.12309349038687771, 'max_val_acc': 0.952850878238678, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
Resultados guardados correctamente en results/efficient_net.json
Experimento terminado para efficientnet_b5.sw_in12k__lots_aug_bn_drp_VAL


In [3]:
file_path = "results/efficient_net.json"
criteria = ["max_val_acc", "best_val_loss"]

for crit in criteria:
    print(f"El mejor modelo según {crit} es {find_best_model(file_path, crit)}")

El mejor modelo según max_val_acc es {'architecture': 'efficientnet_b5.sw_in12k__lots_aug_bn_drp_VAL', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 9, 'best_val_loss': 0.18883399100064538, 'best_train_loss': 0.12309349038687771, 'max_val_acc': 0.952850878238678, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
El mejor modelo según best_val_loss es {'architecture': 'efficientnet_b5.sw_in12k__lots_aug_bn_drp_VAL', 'model_type': 'efficientnet_b5.sw_in12k', 'best_epoch': 9, 'best_val_loss': 0.18883399100064538, 'best_train_loss': 0.12309349038687771, 'max_val_acc': 0.952850878238678, 'test_accuracy': 0.0, 'training_params': {'learning_rate': 0.0003, 'batch_size': 32, 'num_epochs': 50}}
